# Check `PiecewiseQuaternion` Trajectory in Drake

Overarching goal is to take an *empirical* trajectory along $SO(3)$ 
(e.g. recorded from an input device), and compute the exact velocities and accelerations
to recover said trajectory.

This is mean to serve as a means for acausal filtering, to then compare against causal filtering.

Given other checks like those mentioned in `so2_so3_feedback.ipynb`, `sym_so3_notebook.ipynb`, hopefully easy to verify.

## Helper Stuff

In [ ]:
import dataclasses as dc
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np

import trajectories as m
reload(m)
import so2_so3_helpers as m
reload(m)

from trajectories import (
    Sinusoid,
    Mult,
    min_jerk,
    calc_rotational_values,
    make_rot_info_quat_sym,
    So3,
    Coord,
)
from so2_so3_helpers import (
    integrate,
    maxabs,
    flatten,
    unflatten,
    cat,
    split,
    normalize,
    rot2d,
    rot2d_jac,
    so2_angle,
    skew,
    unskew,
    axang,
    axang3,
    axang3_dot,
    to_axang3,
    so3_dist,
    assert_so3,
)

In [ ]:
quat_info = make_rot_info_quat_sym()

## Verify Basics

Reconfirm arbitrary SO(3) trajectory can be recovered via integration along quaternions

In [ ]:
num_r = quat_info.num_rot
num_R = 9

def so3_traj(t):
    # SO(3) trajectory expressed as quaternion, rotation matrix, and
    # ang. vel. + acc.
    As = np.array([0.2, 0.3, 0.4, 0.5])
    func = Sinusoid(Ts=[1.0, 2.0, 3.0, 4.0], T0_ratios=[0, 0.125, 0.25, 0])
    # func = Mult(func, min_jerk)
    dr, rd, rdd = func(t)
    dr *= As
    rd *= As
    rdd *= As
    r = dr + quat_info.r0
    (R, w, wd), (r, rd, rdd) = calc_rotational_values(quat_info, r, rd, rdd)
    return So3(R, w, wd), Coord(r, rd, rdd)

Integrate along both $r(t)$ and $R(t)$ for world-fixed angular velocity. \
Ignore accel for now.

In [ ]:
def to_q(r, R):
    return cat(r, flatten(R))

def from_q(q):
    r, R = split(q, (num_r, num_R))
    R = unflatten(R)
    return r, R

def calc_qd(t, q, *, check=False):
    r, R = from_q(q)
    so, coord = so3_traj(t)
    if check:
        assert_so3(R, tol=1e-4)
        r_err = maxabs(r - coord.r)
        R_err = so3_dist(R, so.R)
        assert r_err < 1e-4
        assert R_err < 1e-2  # blech?
    rd = coord.rd
    w = so.w
    Rd = skew(w) @ R
    qd = to_q(rd, Rd)
    return qd

so0, coord0 = so3_traj(0)
# print(so0)
q0 = to_q(coord0.r, so0.R)

# If this succeeds, then we met our desired tolerances.
integrate(
    q0, calc_qd, tf=3.0, pass_time=True, check=True,
)
print("All matches")